In [1]:
import pandas as pd
import os
from pathlib import Path
import numpy as np

In [2]:
root = Path('/home/anvar/iitp-networks-hw2')

In [3]:
!ls

 edges.csv		  'iitpnetworks2022hw2(4)'   join_networks.ipynb
 iitpnetworks2022hw2	   iitpnetworksha1	     users.csv
'iitpnetworks2022hw2(2)'   iitpnetworksha2
'iitpnetworks2022hw2(3)'  'iitpnetworksha2(1)'


# Collect user info

In [4]:
# df_student = pd.read_excel('/home/anvar/iitp-networks-hw2/iitpnetworksha1/full_table_of_friends.xlsx')
# df_student_edges = pd.read_excel('/home/anvar/iitp-networks-hw2/iitpnetworksha1/mutual_friends.xlsx')
# ids = np.unique(df_student_edges[['friend_1', 'friend_2']].values.reshape(-1))

# df_student[df_student.id.isin(ids)].to_csv('/home/anvar/iitp-networks-hw2/iitpnetworksha1/friends.csv')
# df_student_edges.to_csv('/home/anvar/iitp-networks-hw2/iitpnetworksha1/edges.csv')

In [5]:
friends = [
    '/home/anvar/iitp-networks-hw2/iitpnetworks2022hw2/freinds_list.csv',
    '/home/anvar/iitp-networks-hw2/iitpnetworks2022hw2(3)/DataFrame.csv',
    '/home/anvar/iitp-networks-hw2/iitpnetworksha2/vertices.csv',
    '/home/anvar/iitp-networks-hw2/iitpnetworks2022hw2(4)/df_friends.csv',
    '/home/anvar/iitp-networks-hw2/iitpnetworks2022hw2(2)/friends_info.csv',
    '/home/anvar/iitp-networks-hw2/iitpnetworksha2(1)/info.csv',
    '/home/anvar/iitp-networks-hw2/iitpnetworksha1/friends.csv'
]

In [6]:
dfs = []
for f in friends:
    df = pd.read_csv(f)
    if 'id' in df.columns:
        df.rename(columns={'id':'User id',
                           'first_name':'Name', 
                           'sex': 'Sex', 
                          'bdate':'Birthday',
                          'city':'City',
                          'last_name':'Surname',
                          'university_name':'University'}, inplace=True)
    try:
        df = df[['Birthday', 'City', 'Name', 'Sex', 'University', 'User id']]
    except KeyError:
        df = df[['Birthday', 'Name', 'Sex', 'User id']]
#     print(sorted(list(df.columns)))
    dfs.append(df)
df = pd.concat(dfs).reset_index(drop=True)

def fix_city_name(x):
    if x is np.nan:
        return x
    if 'title' in x:
        return x.split("'")[-2]
    return x

df['City'] = df.City.apply(fix_city_name)

In [7]:
df.head()

,Birthday,City,Name,Sex,University,User id
0,17.4,Saint Petersburg,Egor,2,No,17219
1,12.3,Saint Petersburg,Ekaterina,1,No,37281
2,9.10,Moscow,Alexander,2,No,216456
3,12.6.1985,Saint Petersburg,Kirill,2,No,393551
4,17.6,Saint Petersburg,Elena,1,"БГУ (бывш. БГПИ, БФ НГУ)",727643


# Collect friendship

In [8]:
edges = [
    '/home/anvar/iitp-networks-hw2/iitpnetworks2022hw2/edges.csv',
    '/home/anvar/iitp-networks-hw2/iitpnetworks2022hw2(3)/edges.csv',
    '/home/anvar/iitp-networks-hw2/iitpnetworksha1/edges.csv',
    '/home/anvar/iitp-networks-hw2/iitpnetworksha2/edges.csv',
    '/home/anvar/iitp-networks-hw2/iitpnetworks2022hw2(4)/df_edges.csv',
    '/home/anvar/iitp-networks-hw2/iitpnetworks2022hw2(2)/edges_table.csv',
    '/home/anvar/iitp-networks-hw2/iitpnetworksha2(1)/edges.csv'   
    
]

In [9]:
dfs = []
for edges_path in edges:
    df_edges = pd.read_csv(edges_path)
    if 'friend_1' in df_edges.columns:
        df_edges.rename(columns={'friend_1':'user_id_1', 'friend_2':'user_id_2'}, inplace=True)
    if 'source' in df_edges.columns:
        df_edges.rename(columns={'source':'user_id_1', 'target':'user_id_2'}, inplace=True)
    if '0' in df_edges.columns:
        df_edges.rename(columns={'0':'user_id_1', '1':'user_id_2'}, inplace=True)
    if 'Вершина 1' in df_edges.columns:
        df_edges.rename(columns={'Вершина 1':'user_id_1', 'Вершина 2':'user_id_2'}, inplace=True)
    dfs.append(df_edges[['user_id_1', 'user_id_2']])
df_edges = pd.concat(dfs)
print(df_edges.shape)
df_edges = df_edges.drop_duplicates()
print(df_edges.shape)

(23189, 2)
(20161, 2)


# Shorten Users list

In [10]:
users_with_edge = np.unique(df_edges[['user_id_1', 'user_id_2']].values.reshape(-1))

In [11]:
df = df[df['User id'].isin(users_with_edge)]

# duplicated_ids = df[df.duplicated(subset=['User id'])]['User id']
# df[df['User id'].isin(duplicated_ids)].sort_values('User id').head(20)

df = df.drop_duplicates(subset=['User id']) # likely to loose some meta information about duplicated users, but whatever

In [12]:
df_edges.shape, df.shape

((20161, 2), (1628, 6))

In [13]:
df.to_csv('users.csv')
df_edges.to_csv('edges.csv')